## Execise 1

In this exercise, do the following:
1. Load the dataset used in the time series example - Energy consumption data. You can find it in the notebook "TSA_Example" in Time Series folder in Moodle.
2. Setup a nested MLFlow loop where different modelling experiments can be tracked and the use the dataset in point 1 to experiment and track models. You should do following combinations:
    1. At least 3 model types
    2. At least 3 different feature combinations
    3. At least 3 different options for 3 different hyperparameters
    4. At least 3 different time splits for train test
3. For each option in the combination, you should calculate & log the following in MLFlow:
    1. RMSE
    2. MAE
    3. Plot of actual vs predicted for 1 month data
    4. Plot of actual vs predicted for 1 week of data
    5. All of the combination info in point 2, such as which model, what feature combindation, what hyperparameter, what train test split has been used
4. Turn on MLFlow UI and track your experiments

<hr>
<h4>Task 1 Load the dataset used in the time series example - Energy consumption data. You can find it in the notebook "TSA_Example" in Time Series folder in Moodle.</h4>

In [1]:
import os
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import mlflow
import mlflow.sklearn


path = kagglehub.dataset_download("robikscube/hourly-energy-consumption")
files = os.listdir(path)
print(files)
df = pd.read_csv(path + "/PJME_hourly.csv")
display(df.head())
print(df.shape)
print(df.columns.tolist())

['AEP_hourly.csv', 'COMED_hourly.csv', 'DAYTON_hourly.csv', 'DEOK_hourly.csv', 'DOM_hourly.csv', 'DUQ_hourly.csv', 'EKPC_hourly.csv', 'est_hourly.paruqet', 'FE_hourly.csv', 'NI_hourly.csv', 'PJME_hourly.csv', 'PJMW_hourly.csv', 'pjm_hourly_est.csv', 'PJM_Load_hourly.csv']


,Datetime,PJME_MW
0,2002-12-31 01:00:00,26498.0
1,2002-12-31 02:00:00,25147.0
2,2002-12-31 03:00:00,24574.0
3,2002-12-31 04:00:00,24393.0
4,2002-12-31 05:00:00,24860.0


(145366, 2)
['Datetime', 'PJME_MW']


In [2]:
df.dtypes

Datetime     object
PJME_MW     float64
dtype: object

Set Datetime column as datatype "Datetime"

In [3]:
df['Datetime']=pd.to_datetime(df['Datetime'])
df.dtypes

Datetime    datetime64[ns]
PJME_MW            float64
dtype: object

Look for dublicates and nulls

In [4]:
print("Duplicate count:", df.duplicated(['Datetime']).sum())
print("Null values count:", df.isnull().sum())

Duplicate count: 4
Null values count: Datetime    0
PJME_MW     0
dtype: int64


In [5]:
df = df.drop_duplicates(subset=['Datetime'])
print("Duplicate count:", df.duplicated(['Datetime']).sum())

Duplicate count: 0


Lets put Datetime as index and sort it

In [6]:
# Set index 
df = df.sort_values("Datetime").set_index("Datetime")
df = df.asfreq("h")
df.head()

,PJME_MW
Datetime,
2002-01-01 01:00:00,30393.0
2002-01-01 02:00:00,29265.0
2002-01-01 03:00:00,28357.0
2002-01-01 04:00:00,27899.0
2002-01-01 05:00:00,28057.0


In [7]:
df.index.name       
print("Index name:", df.index.name)
print("DataFrame shape:", df.shape)

Index name: Datetime
DataFrame shape: (145392, 1)


Since the dataset only had two columns, feature(timestamp) and target(energy). Lets do feature engineering to the timestamp. Like month, weekend, day of week, hour, day/night, and rolling, lag. 

In [8]:
print("Years present:", sorted(df.index.year.unique())) # Since many years are present lets add them as features as well.

Years present: [2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]


In [9]:
df["hour"] = df.index.hour
df["day_of_week"] = df.index.dayofweek
df["month"] = df.index.month
df["year"] = df.index.year
df["is_weekend"] = df.index.dayofweek >= 5
df["is_night"] = (df.index.hour < 6) | (df.index.hour >= 22)

display(df.head())


,PJME_MW,hour,day_of_week,month,year,is_weekend,is_night
Datetime,,,,,,,
2002-01-01 01:00:00,30393.0,1,1,1,2002,False,True
2002-01-01 02:00:00,29265.0,2,1,1,2002,False,True
2002-01-01 03:00:00,28357.0,3,1,1,2002,False,True
2002-01-01 04:00:00,27899.0,4,1,1,2002,False,True
2002-01-01 05:00:00,28057.0,5,1,1,2002,False,True


In [10]:
target = "PJME_MW"

df["lag_1"] = df[target].shift(1)     # 1 hour lag
df["lag_2"] = df[target].shift(2)     # 2 hour lag
df["lag_3"] = df[target].shift(3)     # 3 hour lag
df["lag_24"] = df[target].shift(24)   # 1 day lag
df["lag_48"] = df[target].shift(48)   # 2 day lag
df["lag_168"] = df[target].shift(168) # 7 day lag

df["rolling_mean_1"] = df[target].rolling(window=1).mean()   # 1 hour rolling mean
df["rolling_mean_3"] = df[target].rolling(window=3).mean()   # 3 hour rolling mean
df["rolling_mean_6"] = df[target].rolling(window=6).mean()   # 6 hour rolling mean
df["rolling_mean_24"] = df[target].rolling(window=24).mean()  # 1 day rolling mean
df["rolling_mean_48"] = df[target].rolling(window=48).mean()  # 2 day rolling mean
df["rolling_mean_168"] = df[target].rolling(window=168).mean() # 7 day rolling mean
df["rolling_mean_720"] = df[target].rolling(window=720).mean() # 30 day rolling mean


df["rolling_std_3"] = df[target].rolling(window=3).std()   # 3 hour rolling std
df["rolling_std_6"] = df[target].rolling(window=6).std()   # 6 hour rolling std
df["rolling_std_24"] = df[target].rolling(window=24).std() # 1 day rolling std
df["rolling_std_48"] = df[target].rolling(window=48).std() # 2 day rolling std
df["rolling_std_168"] = df[target].rolling(window=168).std()# 7 day rolling std
df["rolling_std_720"] = df[target].rolling(window=720).std()# 30 day rolling std
df.dropna()

,PJME_MW,hour,day_of_week,month,year,is_weekend,is_night,lag_1,lag_2,lag_3,...,rolling_mean_24,rolling_mean_48,rolling_mean_168,rolling_mean_720,rolling_std_3,rolling_std_6,rolling_std_24,rolling_std_48,rolling_std_168,rolling_std_720
Datetime,,,,,,,,,,,,,,,,,,,,,
2002-01-31 00:00:00,25332.0,0,3,1,2002,False,True,27809.0,30371.0,32107.0,...,28599.375000,28786.062500,28934.113095,31090.741667,2519.619482,3286.475600,4153.523992,3853.209386,3600.585976,3963.736702
2002-01-31 01:00:00,23559.0,1,3,1,2002,False,True,25332.0,27809.0,30371.0,...,28628.166667,28775.125000,28924.059524,31081.250000,2134.695841,3798.938189,4114.277058,3867.558769,3613.275643,3973.580071
2002-01-31 02:00:00,22734.0,2,3,1,2002,False,True,23559.0,25332.0,27809.0,...,28661.583333,28765.270833,28914.869048,31072.179167,1327.513842,3766.158290,4060.429908,3882.620916,3627.100955,3985.169485
2002-01-31 03:00:00,22363.0,3,3,1,2002,False,True,22734.0,23559.0,25332.0,...,28694.958333,28755.541667,28906.571429,31063.854167,612.193052,3169.541965,4002.414646,3898.363301,3640.540734,3997.092253
2002-01-31 04:00:00,22487.0,4,3,1,2002,False,True,22363.0,22734.0,23559.0,...,28743.583333,28748.395833,28899.583333,31056.337500,188.867679,2146.412697,3915.056887,3909.753846,3651.779424,4008.125642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-02 20:00:00,44057.0,20,3,8,2018,False,False,45641.0,46760.0,46816.0,...,39902.916667,39228.375000,35900.238095,36195.288889,1358.149844,1193.547220,6368.579836,6846.026745,6755.525713,7276.625275
2018-08-02 21:00:00,43256.0,21,3,8,2018,False,False,44057.0,45641.0,46760.0,...,39789.208333,39297.500000,35892.583333,36189.683333,1213.732672,1589.298556,6278.997878,6870.053272,6746.397424,7269.616216
2018-08-02 22:00:00,41552.0,22,3,8,2018,False,True,43256.0,44057.0,45641.0,...,39683.291667,39358.791667,35886.119048,36184.191667,1279.338501,2098.223312,6224.428358,6876.909400,6740.413014,7264.059984


Since we want to predict the target at the next row, lets make a coloumn for target+1

In [11]:
df["y"] = df["PJME_MW"].shift(-1)  # Target for next hour
df = df.dropna()
print(df.shape)
df[["PJME_MW","y"]].head()

(123042, 27)


,PJME_MW,y
Datetime,,
2002-01-31 00:00:00,25332.0,23559.0
2002-01-31 01:00:00,23559.0,22734.0
2002-01-31 02:00:00,22734.0,22363.0
2002-01-31 03:00:00,22363.0,22487.0
2002-01-31 04:00:00,22487.0,23087.0


In [12]:
print(df.columns.to_list)

<bound method IndexOpsMixin.tolist of Index(['PJME_MW', 'hour', 'day_of_week', 'month', 'year', 'is_weekend',
       'is_night', 'lag_1', 'lag_2', 'lag_3', 'lag_24', 'lag_48', 'lag_168',
       'rolling_mean_1', 'rolling_mean_3', 'rolling_mean_6', 'rolling_mean_24',
       'rolling_mean_48', 'rolling_mean_168', 'rolling_mean_720',
       'rolling_std_3', 'rolling_std_6', 'rolling_std_24', 'rolling_std_48',
       'rolling_std_168', 'rolling_std_720', 'y'],
      dtype='object')>


Encoding

In [13]:
categorical_columns = ["hour", "day_of_week", "month", "year", "is_weekend", "is_night"]
df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

encoded_calendar_columns = [i for i in df_encoded.columns if any(i.startswith(prefix) for prefix in
["hour_","day_of_week_","month_","year_","is_weekend_","is_night_"])]
print(encoded_calendar_columns)

lags = ["lag_1", "lag_2", "lag_3", "lag_24", "lag_48", "lag_168"]

rolling_means = ["rolling_mean_1", "rolling_mean_3", "rolling_mean_6", "rolling_mean_24", "rolling_mean_48", "rolling_mean_168"]

rolling_stds = ["rolling_std_3", "rolling_std_6", "rolling_std_24", "rolling_std_48", "rolling_std_168", "rolling_std_720"]

print(f"All columns: {df.columns.to_list()}\n \n Only calendar columns: {encoded_calendar_columns}")

['hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 'year_2018', 'is_weekend_True', 'is_night_True']
All columns: ['PJME_MW', 'hour', 'day_of_week', 'month', 'year', 'is_weekend', 'is_night', 'lag_1', 'lag_2', 'lag_3', 'lag_24', 'lag_48', 'lag_168', 'rolling_mean_1', 'rolling_mean_3', 'rolling_mean_6', 'rolling_mean_24', 'rolling_mean_48', 'rolling_mean_168', 'rolling_mean_720', 'rol

<hr>
<h4>Task 2 Setup a nested MLFlow loop where different modelling experiments can be tracked and the use the dataset in point 1 to experiment and track models. You should do following combinations:
 <ol>
    <li>At least 3 model types</li>
    <li>At least 3 different feature combinations</li>
    <li>At least 3 different options for 3 different hyperparameters</li>
    <li>At least 3 different time splits for train test</li>
 </ol>
    </h4>

<h3> 3 different feature combinations</h3>

In [14]:
lags = ["lag_1", "lag_2", "lag_3", "lag_24", "lag_48", "lag_168"]
rolling_means = ["rolling_mean_1", "rolling_mean_3", "rolling_mean_6", "rolling_mean_24", "rolling_mean_48", "rolling_mean_168"]
rolling_stds = ["rolling_std_3", "rolling_std_6", "rolling_std_24", "rolling_std_48", "rolling_std_168", "rolling_std_720"]

feature_sets1 = encoded_calendar_columns + lags

feature_sets2 = encoded_calendar_columns + lags + rolling_means

feature_sets3 = encoded_calendar_columns + lags + rolling_means + rolling_stds

<h3>3 Different time splits for train test</h3>

In [15]:
cutoffs = [
    pd.Timestamp("2017-01-01"),
    pd.Timestamp("2017-07-01"),
    pd.Timestamp("2018-01-01"),
]

first_cutoff = cutoffs[0]
second_cutoff = cutoffs[1]
third_cutoff = cutoffs[2]

<h3>3 different options for 3 different hyperparameters</h3>

In [21]:
parameter_grid = {
    "Linear Regression": [{}],
    "Ridge Regression": [
        {"alpha": 0.1, "random_state": 2},
        {"alpha": 1.0, "random_state": 2},
        {"alpha": 10.0, "random_state": 2}
    ],
    "Random Forest": [
       {"n_estimators": 200, "max_depth": None, "min_samples_split": 2, "random_state": 2},
       {"n_estimators": 400, "max_depth": 20,   "min_samples_split": 2, "random_state": 2},
       {"n_estimators": 800, "max_depth": 12,   "min_samples_split": 3, "random_state": 2},
    ],
    "XGBoost": [
       {"n_estimators": 200, "max_depth": 4, "learning_rate": 0.1, "random_state": 2},
       {"n_estimators": 600, "max_depth": 6, "learning_rate": 0.05, "random_state": 2},
       {"n_estimators": 900, "max_depth": 8, "learning_rate": 0.2, "random_state": 2}
    ]
}


<h3>Training loop</h3>

In [ ]:
mlflow.set_experiment("energy_prediction")

models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Random Forest": RandomForestRegressor(),
    "XGBoost": XGBRegressor()
}

feature_sets = {
    "calendar+lags": feature_sets1,
    "calendar+lags+rolling_means": feature_sets2,
    "calendar+lags+rolling_means+rolling_stds": feature_sets3
}

for cutoff in cutoffs:
    train_mask = df_encoded.index < cutoff
    test_mask = df_encoded.index >= cutoff

    for fset_name, features in feature_sets.items():
        X_train, y_train = df_encoded.loc[train_mask, features], df_encoded.loc[train_mask, "y"]
        X_test, y_test = df_encoded.loc[test_mask, features], df_encoded.loc[test_mask, "y"]

        for model_name, param_list in parameter_grid.items():
            for params in param_list:
                if model_name == "Linear Regression":
                    model = LinearRegression(**params)
                elif model_name == "Ridge Regression":
                    model = Ridge(**params)
                elif model_name == "Random Forest":
                    model = RandomForestRegressor(**params)
                elif model_name == "XGBoost":
                    model = XGBRegressor(**params)
                else:
                    continue

                print(f"Training {model_name} with feature set {fset_name} on cutoff {cutoff.date()}...")
                with mlflow.start_run(run_name=f"{model_name}_{fset_name}_{cutoff}"):
                    model.fit(X_train, y_train)
                    preds = model.predict(X_test)

                    mse = mean_squared_error(y_test, preds)
                    mae = mean_absolute_error(y_test, preds)

                    mlflow.log_param("cutoffs", cutoff.date())
                    mlflow.log_param("feature_set", fset_name)
                    mlflow.log_param("model", model_name)
                    #mlflow.log_metric("RMSE", rmse)
                    mlflow.log_metric("MSE", mse)
                    mlflow.log_metric("MAE", mae)

                    print(f"{cutoff.date()}, {fset_name}, {model_name}, {mae}")


Traceback (most recent call last):
  File "c:\Users\sebas\AppData\Local\Programs\Python\Python310\lib\site-packages\mlflow\store\tracking\file_store.py", line 366, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "c:\Users\sebas\AppData\Local\Programs\Python\Python310\lib\site-packages\mlflow\store\tracking\file_store.py", line 464, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\Users\sebas\AppData\Local\Programs\Python\Python310\lib\site-packages\mlflow\store\tracking\file_store.py", line 1634, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\Users\sebas\AppData\Local\Programs\Python\Python310\lib\site-packages\mlflow\store\tracking\file_store.py", line 1627, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\Users\sebas\AppData\Local\Programs\Python\Python310\lib\site-packages\mlflow\utils\yaml_utils.py", line 107, in read_yaml
    r

Training Linear Regression with feature set calendar+lags on cutoff 2017-01-01...
2017-01-01, calendar+lags, Linear Regression, 777.9168860937226
Training Ridge Regression with feature set calendar+lags on cutoff 2017-01-01...
2017-01-01, calendar+lags, Ridge Regression, 777.91247204645
Training Ridge Regression with feature set calendar+lags on cutoff 2017-01-01...
2017-01-01, calendar+lags, Ridge Regression, 777.8728791081096
Training Ridge Regression with feature set calendar+lags on cutoff 2017-01-01...
2017-01-01, calendar+lags, Ridge Regression, 777.4931763154617
Training Random Forest with feature set calendar+lags on cutoff 2017-01-01...


3. For each option in the combination, you should calculate & log the following in MLFlow:
    1. RMSE
    2. MAE
    3. Plot of actual vs predicted for 1 month data
    4. Plot of actual vs predicted for 1 week of data
    5. All of the combination info in point 2, such as which model, what feature combindation, what hyperparameter, what train test split has been used


4. Turn on MLFlow UI and track your experiments